# Задание 1

    Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [79]:
ratings = pd.read_csv('ml-latest-small/ratings.csv', usecols=[0,3], dtype=int)  # читаем ИД и таймстампы (сразу в int)
ratings.head()

users = ratings.groupby('userId').count()  # группируем, фильтруем, переименовываем колонку
users = users.loc[users.timestamp > 100 ].reset_index()
users.rename(columns={'timestamp': 'visits'}, inplace=True)

def lifetime(row):  # функция для получения времени жизни в секундах - в датасете есть разница около часа
    return (row['ld']-row['fd']).seconds

res = pd.merge(users, ratings, on='userId', how='inner')  # мержим первую колонку - минимальных таймстампов
res = res.groupby('userId').min()
res.rename(columns={'timestamp':'first'}, inplace=True)  # переименовали
res = pd.merge(res, ratings, on='userId', how='inner')  # вторая колонка - максимальные
res = res.groupby('userId').max()
res.rename(columns={'timestamp':'last'}, inplace=True)  # переименовали
res['fd'] = pd.to_datetime(res['first'], unit='s')  # преобразовали в дэйттайм
res['ld'] = pd.to_datetime(res['last'], unit='s')
#res['diff'] = res['ld']-res['fd']
res['lifetime'] = res.apply(lifetime, axis=1)  # применили функцию
avrg = res['lifetime'].mean()  # среднее по колонке
print(f'среднее время жизни {avrg:.0f} секунд')


среднее время жизни 32098 c


# Задание 2

    Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:

    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [80]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [95]:
# собираем полную таблицу, из нее выделим облегченную без адресов
full_table = pd.merge(client_base, rzd, on='client_id', how='left')
full_table = pd.merge(full_table, auto, on='client_id', how='left')
full_table = pd.merge(full_table, air, on='client_id', how='left')
full_table.loc[pd.isnull(full_table.rzd_revenue), 'rzd_revenue'] = 0
full_table.loc[pd.isnull(full_table.auto_revenue), 'auto_revenue'] = 0
full_table.loc[pd.isnull(full_table.air_revenue), 'air_revenue'] = 0

light_table = full_table.loc[:, ['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']]
print(light_table.head(10))
print(full_table.head(10))

   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0
   client_id          address  rzd_revenue  auto_revenue  air_revenue
0        111  Комсомольская 4       1093.0           0.0          0.0
1        112   Энтузиастов 8а       2810.0           0.0          0.0
2        113  Левобережная 1а      10283.0       57483.0          0.0
3        114          Мира 14       5774.0          83.0          0.0
4        115        ЗЖБИиДК 1        981.0         912.0         81.0
5        116    Строителей 18          0.0        4834.0          4.0
6        117  Панфиловская 33    

# Задание 3

    В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

    У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
    Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
    Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


In [ ]:
Архитектура: прежде всего ничего не сказано о формате последовательности координат - очевидно, формат должен также содержать timestamp
    также потребуется пара функций сравнения координат и временных интервалов
1. очевидно, речь идет о UserID и timestamp. если под "одно-два числа" подразумеваются координаты - ближайшие по времени события
2. никак, если изначально выбирать правильную архитектуру и алгоритмы. удаленность координат может означать одновременное использование учетки на нескольких устройствах - но если использовать ID сессии - проблема решается
3. если речь о тотальной слежке - то аналитики очень много - ареал обитания, распорядок дня, характер передвижения и транспорт - очень легко (без сверхестественной аналитики) разбиваются категории работающих/неработающих, автомобилистов, туристов, офисных работников и т.д.

Ответ на вопрос: Как бы вы добавили эти сведения в таблицу визитов и покупок?
    в таблицу визитов - никак, мы и так знаем где нам нанесли визит и где совершили физическую покупку.
    при необходимости геопозиционирования при онлайн событиях - рекомендуется держать отдельную табличку с этими событиями и их координатами, т.к. как правило стоимость процессорного времени на джоины с логами куда дороже стоимости хранения таблички размером в гигабайт (а это очень-очень много покупок)